<h1 align=center> TOP14 2022-2023's final ranking "prediction" with machine learning </h1>

In [1]:
# Import libraries
import pandas as pd
from skimage import io
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV

<h2 align="center"> Training and testing the model </h2>

In [2]:
#Import data
df = pd.read_excel('data/Résultats.xlsx')
df.head()

,Saison,Année,Journée,Equipe 1,Equipe 2,Score Equipe 1,Score Equipe 2,Essais accordés Equipe 1,Essais accordés Equipe 2,Possession de la balle Equipe 1,...,Ballons joués au pied Equipe 1,Ballons joués au pied Equipe 2,Ballons passés Equipe 1,Ballons passés Equipe 2,Equipe victorieuse,Victoire domicile,Bonus offensif vainqueur,Equipe défaite,Bonus défensif,Bonus offensif perdant
0,2019-2020,2019,1,Castres Olympique,Montpellier Hérault Rugby,26,25,1,3,0.5,...,18,14,96,131,Castres Olympique,True,False,Montpellier Hérault Rugby,True,False
1,2019-2020,2019,1,LOU Rugby,Stade Français Paris,43,9,7,0,0.5,...,25,13,163,126,LOU Rugby,True,True,Stade Français Paris,False,False
2,2019-2020,2019,1,Racing 92,Aviron Bayonnais,17,24,2,2,0.5,...,19,25,176,151,Aviron Bayonnais,False,False,Racing 92,False,False
3,2019-2020,2019,1,Section Paloise,CA Brive,33,14,4,2,0.5,...,19,11,75,167,Section Paloise,True,False,CA Brive,False,False
4,2019-2020,2019,1,SU Agen,RC Toulon,25,44,3,6,0.5,...,11,21,101,132,RC Toulon,False,True,SU Agen,False,False


In [3]:
# Cleaning data
#ML algorithms can only work on numérical data int64 / float64 and cannot use object or bool values.
df.dtypes

# Drop the 'Saison' column.
df = df.drop(['Saison'], axis=1)

# Assign a number to each team to convert the object type into int64:
df["Num equipe 1"] = df["Equipe 1"].astype("category").cat.codes
df["Num equipe 2"] = df["Equipe 2"].astype("category").cat.codes

# Convert bool into 0/1 values.
df.replace({False: 0, True: 1}, inplace=True)

# Drop NaN values.
df = df.dropna(axis=0)

df

,Année,Journée,Equipe 1,Equipe 2,Score Equipe 1,Score Equipe 2,Essais accordés Equipe 1,Essais accordés Equipe 2,Possession de la balle Equipe 1,Possession de la balle Equipe 2,...,Ballons passés Equipe 1,Ballons passés Equipe 2,Equipe victorieuse,Victoire domicile,Bonus offensif vainqueur,Equipe défaite,Bonus défensif,Bonus offensif perdant,Num equipe 1,Num equipe 2
0,2019,1,Castres Olympique,Montpellier Hérault Rugby,26,25,1,3,0.5,0.5,...,96,131,Castres Olympique,1,0,Montpellier Hérault Rugby,1,0,4,6
1,2019,1,LOU Rugby,Stade Français Paris,43,9,7,0,0.5,0.5,...,163,126,LOU Rugby,1,1,Stade Français Paris,0,0,5,11
2,2019,1,Racing 92,Aviron Bayonnais,17,24,2,2,0.5,0.5,...,176,151,Aviron Bayonnais,0,0,Racing 92,0,0,8,1
3,2019,1,Section Paloise,CA Brive,33,14,4,2,0.5,0.5,...,75,167,Section Paloise,1,0,CA Brive,0,0,10,3
4,2019,1,SU Agen,RC Toulon,25,44,3,6,0.5,0.5,...,101,132,RC Toulon,0,1,SU Agen,0,0,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022,26,Racing 92,RC Toulon,21,16,2,1,0.5,0.5,...,118,92,Racing 92,1,0,RC Toulon,1,0,8,7
479,2022,26,Section Paloise,Castres Olympique,16,26,1,2,0.5,0.5,...,142,84,Castres Olympique,0,0,Section Paloise,0,0,10,4
480,2022,26,Stade Français Paris,CA Brive,17,33,2,3,0.5,0.5,...,80,92,CA Brive,0,0,Stade Français Paris,0,0,11,3
481,2022,26,Stade Toulousain,Biarritz Olympique PB,80,7,12,1,0.5,0.5,...,146,105,Stade Toulousain,1,1,Biarritz Olympique PB,0,0,13,2


In [4]:
# Select the features that will be used to train the model.
features = df[[ 
        "Possession dans son camp Equipe 1",
        "Possession dans son camp Equipe 2",
        "Possession dans le camp adverse Equipe 1",
        "Possession dans le camp adverse Equipe 2",
        "Possession 22m adverses Equipe 1", 
        "Possession 22m adverses Equipe 2", 
        "Mêlées obtenues Equipe 1",
        "Mêlées obtenues Equipe 2",
        "Mêlées gagnées Equipe 1",
        "Mêlées gagnées Equipe 2",
        "Mêlées perdues Equipe 1",
        "Mêlées perdues Equipe 2",
        "Touches obtenues Equipe 1",
        "Touches obtenues Equipe 2",
        "Touches gagnées sur son propre lancer Equipe 1",
        "Touches gagnées sur son propre lancer Equipe 2",
        "Touches gagnées sur lancer adverse Equipe 1",
        "Touches gagnées sur lancer adverse Equipe 2",
        "En-avant commis Equipe 1",
        "En-avant commis Equipe 2",
        "Pénalités réussies Equipe 1",
        "Pénalités réussies Equipe 2",
        "Pénalités concédées Equipe 1",
        "Pénalités concédées Equipe 2",
        "Plaquages réussis Equipe 1",
        "Plaquages réussis Equipe 2",
        "Plaquages offensifs réussis Equipe 1",
        "Plaquages offensifs réussis Equipe 2",
        "Plaquages manqués Equipe 1",
        "Plaquages manqués Equipe 2",
        "Ballons joués au pied Equipe 1",
        "Ballons joués au pied Equipe 2",
        "Ballons passés Equipe 1",
        "Ballons passés Equipe 2"
        ]]

# Define the targets the model will have to predict.
target_win = df["Victoire domicile"]
target_try_1 = df["Essais accordés Equipe 1"]
target_try_2 = df["Essais accordés Equipe 2"]
target_score_1 = df["Score Equipe 1"]
target_score_2 = df["Score Equipe 2"]

# Divide data into training and test sets.
X_train, X_test, y_train_win, y_test_win, y_train_try_1, y_test_try_1, y_train_try_2, y_test_try_2, y_train_score_1, y_test_score_1, y_train_score_2, y_test_score_2 = train_test_split(
    features, target_win, target_try_1, target_try_2, target_score_1, target_score_2, test_size=0.2, random_state=42
)

# Data normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Use the model RandomForest.
#Use the model RandomForestClassifier for binary target variables: win or not.
model_win = RandomForestClassifier(n_estimators= 100, random_state=42)

#Use the model RandomForestRegressor for continuous target variables.
model_try_1 = RandomForestRegressor(n_estimators= 100, random_state=42)
model_try_2 = RandomForestRegressor(n_estimators= 100, random_state=42)
model_score_1 = RandomForestRegressor(n_estimators= 100, random_state=42)
model_score_2 = RandomForestRegressor(n_estimators= 100, random_state=42)

# Train the models.
model_win.fit(X_train_scaled, y_train_win)
model_try_1.fit(X_train_scaled, y_train_try_1)
model_try_2.fit(X_train_scaled, y_train_try_2)
model_score_1.fit(X_train_scaled, y_train_score_1)
model_score_2.fit(X_train_scaled, y_train_score_2)

# Predictions for the test set.
y_pred_win = model_win.predict(X_test_scaled)
y_pred_try_1 = model_try_1.predict(X_test_scaled)
y_pred_try_2 = model_try_2.predict(X_test_scaled)
y_pred_score_1 = model_score_1.predict(X_test_scaled)
y_pred_score_2 = model_score_2.predict(X_test_scaled)

# Evaluation of the win prediction model using a measure of accuracy.
accuracy_win = accuracy_score(y_test_win, y_pred_win)
print(f"Accuracy for Win: {accuracy_win * 100:.2f}%")

# Evaluation of the tries and score prediction models using a measure of accuracy using the mean_squared_error for regressions.
mse_try_1 = mean_squared_error(y_test_try_1, y_pred_try_1)
mse_try_2 = mean_squared_error(y_test_try_2, y_pred_try_2)
mse_score_1 = mean_squared_error(y_test_score_1, y_pred_score_1)
mse_score_2 = mean_squared_error(y_test_score_2, y_pred_score_2)

print(f"Mean Squared Error for Essais Equipe 1: {mse_try_1}")
print(f"Mean Squared Error for Essais Equipe 2: {mse_try_2}")
print(f"Mean Squared Error for Score Equipe 1: {mse_score_1}")
print(f"Mean Squared Error for Score Equipe 2: {mse_score_2}")


Accuracy for Win: 69.07%
Mean Squared Error for Essais Equipe 1: 2.711343298969072
Mean Squared Error for Essais Equipe 2: 1.7081123711340205
Mean Squared Error for Score Equipe 1: 108.77020309278352
Mean Squared Error for Score Equipe 2: 59.21987319587629


- The win prediction model has an accuracy of 69.07%. In other words, it correctly predicts whether a team will win or lose around 69 times out of 100.

- The Mean Squared Error (MSE) related to the prediction of the number of tries are relatively low. This is satisfying for the needs.

- Whereas the Mean Squared Error (MSE) related to the prediction of the scores are very high. This is not satisfying enough, the hyperparameters could be change to enhance the model:

In [5]:
# Define values for the hyperparameters to be tested.
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


new_model_score_1 = RandomForestRegressor(random_state=42)

# Use GridSearchCV to find the best hyperparameters.
grid_search_score_1 = GridSearchCV(new_model_score_1, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_score_1.fit(X_train_scaled, y_train_score_1)

print("Best hyperparameters for Score Equipe 1:", grid_search_score_1.best_params_)

Best hyperparameters for Score Equipe 1: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


In [6]:
new_model_score_2 = RandomForestRegressor(random_state=42)

# Use GridSearchCV to find the best hyperparameters
grid_search_score_2 = GridSearchCV(new_model_score_2, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_score_2.fit(X_train_scaled, y_train_score_2)

print("Best hyperparameters for Score Equipe 2:", grid_search_score_2.best_params_)

Best hyperparameters for Score Equipe 2: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [7]:
#Use the model RandomForestRegressor for continuous target variables, with the optimized hyperparameters.
new_model_score_1 = RandomForestRegressor(max_depth= 10, min_samples_leaf= 2, min_samples_split= 5, n_estimators= 200, random_state=42)
new_model_score_2 = RandomForestRegressor(max_depth= None, min_samples_leaf= 1, min_samples_split= 5, n_estimators= 200, random_state=42)

# Retrain the models.
new_model_score_1.fit(X_train_scaled, y_train_score_1)
new_model_score_2.fit(X_train_scaled, y_train_score_2)

# Predictions for the test set.
new_y_pred_score_1 = model_score_1.predict(X_test_scaled)
new_y_pred_score_2 = model_score_2.predict(X_test_scaled)

# Evaluation of the new score prediction models using a measure of accuracy using the mean_squared_error for regressions.
new_mse_score_1 = mean_squared_error(y_test_score_1, new_y_pred_score_1)
new_mse_score_2 = mean_squared_error(y_test_score_2, new_y_pred_score_2)

print(f"Mean Squared Error for Score Equipe 1: {new_mse_score_1}")
print(f"Mean Squared Error for Score Equipe 2: {new_mse_score_2}")

Mean Squared Error for Score Equipe 1: 108.77020309278352
Mean Squared Error for Score Equipe 2: 59.21987319587629


<h3> The model is a bit enhanced but still not so accurate to predict the score. In the following part, the "old" model will be used. </h3>

<h2 align="center"> Making the predictions </h2>

In [8]:
#Select the dataFrame containing the matches for the season 2022-2023.
df_predictions = pd.read_excel("data/Prédictions.xlsx")
df_predictions = df_predictions.dropna(axis=0)

# Select the features that will be used to feed the model, so that he predicts correctly the targets.
features_predictions = df_predictions[[ 
    "Possession dans son camp Equipe 1",
    "Possession dans son camp Equipe 2",
    "Possession dans le camp adverse Equipe 1",
    "Possession dans le camp adverse Equipe 2",
    "Possession 22m adverses Equipe 1", 
    "Possession 22m adverses Equipe 2", 
    "Mêlées obtenues Equipe 1",
    "Mêlées obtenues Equipe 2",
    "Mêlées gagnées Equipe 1",
    "Mêlées gagnées Equipe 2",
    "Mêlées perdues Equipe 1",
    "Mêlées perdues Equipe 2",
    "Touches obtenues Equipe 1",
    "Touches obtenues Equipe 2",
    "Touches gagnées sur son propre lancer Equipe 1",
    "Touches gagnées sur son propre lancer Equipe 2",
    "Touches gagnées sur lancer adverse Equipe 1",
    "Touches gagnées sur lancer adverse Equipe 2",
    "En-avant commis Equipe 1",
    "En-avant commis Equipe 2",
    "Pénalités réussies Equipe 1",
    "Pénalités réussies Equipe 2",
    "Pénalités concédées Equipe 1",
    "Pénalités concédées Equipe 2",
    "Plaquages réussis Equipe 1",
    "Plaquages réussis Equipe 2",
    "Plaquages offensifs réussis Equipe 1",
    "Plaquages offensifs réussis Equipe 2",
    "Plaquages manqués Equipe 1",
    "Plaquages manqués Equipe 2",
    "Ballons joués au pied Equipe 1",
    "Ballons joués au pied Equipe 2",
    "Ballons passés Equipe 1",
    "Ballons passés Equipe 2"
]]

# Normalise prediction data.
features_predictions_scaled = scaler.transform(features_predictions)

# Predict the tries and the score for each match.
df_predictions['Essais Equipe 1'] = model_try_1.predict(features_predictions_scaled)
df_predictions['Essais Equipe 2'] = model_try_2.predict(features_predictions_scaled)
df_predictions['Score Equipe 1'] = model_score_1.predict(features_predictions_scaled)
df_predictions['Score Equipe 2'] = model_score_2.predict(features_predictions_scaled)

# Predict home wins for each match.
df_predictions['Prediction'] = model_win.predict(features_predictions_scaled)

# Determine the number of points each team has at the end of the championship. 
df_predictions['Equipe victorieuse'] = df_predictions.apply(lambda row: row['Equipe 1'] if row['Prediction'] == 1 else row['Equipe 2'], axis=1)
df_predictions['Equipe défaite'] = df_predictions.apply(lambda row: row['Equipe 1'] if row['Prediction'] == 0 else row['Equipe 2'], axis=1)
df_predictions['Bonus défensif'] = df_predictions.apply(lambda row: True if (abs(row['Score Equipe 1'] - row['Score Equipe 2']) <= 5) else False, axis=1)
df_predictions['Bonus offensif vainqueur'] = df_predictions.apply(lambda row: True if (((row['Essais Equipe 1'] - row['Essais Equipe 2']) >= 3) and row['Prediction'] == 1) or (((row['Essais Equipe 2'] - row['Essais Equipe 1']) >= 3) and row['Prediction'] == 0) else False, axis=1)
df_predictions['Bonus offensif perdant'] = df_predictions.apply(lambda row: True if (((row['Essais Equipe 1'] - row['Essais Equipe 2']) >= 3) and row['Prediction'] == 0) or (((row['Essais Equipe 2'] - row['Essais Equipe 1']) >= 3) and row['Prediction'] == 1) else False, axis=1)

df_predictions

,Saison,Année,Journée,Equipe 1,Equipe 2,Possession de la balle Equipe 1,Possession de la balle Equipe 2,Occupation Equipe 1,Occupation Equipe 2,Possession dans son camp Equipe 1,...,Essais Equipe 1,Essais Equipe 2,Score Equipe 1,Score Equipe 2,Prediction,Equipe victorieuse,Equipe défaite,Bonus défensif,Bonus offensif vainqueur,Bonus offensif perdant
0,2022-2023,2022,1,Racing 92,Castres Olympique,0.50,0.50,0.44,0.56,0.70,...,2.17,0.98,22.44,21.09,1,Racing 92,Castres Olympique,True,False,False
1,2022-2023,2022,1,CA Brive,LOU Rugby,0.50,0.50,0.40,0.60,0.38,...,2.13,5.53,18.63,35.52,0,LOU Rugby,CA Brive,False,True,False
2,2022-2023,2022,1,RC Toulon,Aviron Bayonnais,0.50,0.50,0.59,0.41,0.46,...,2.34,2.25,28.48,20.54,1,RC Toulon,Aviron Bayonnais,False,False,False
3,2022-2023,2022,1,Section Paloise,USA Perpignan,0.50,0.50,0.45,0.55,0.39,...,2.18,1.60,27.74,15.18,1,Section Paloise,USA Perpignan,False,False,False
4,2022-2023,2022,1,Stade Français Paris,ASM Clermont,0.50,0.50,0.53,0.47,0.92,...,2.59,1.50,28.12,20.24,1,Stade Français Paris,ASM Clermont,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2022-2023,2023,26,LOU Rugby,Aviron Bayonnais,0.56,0.44,0.51,0.49,0.48,...,4.31,2.09,34.71,17.93,1,LOU Rugby,Aviron Bayonnais,False,False,False
178,2022-2023,2023,26,RC Toulon,Union Bordeaux-Bègles,0.49,0.51,0.47,0.53,0.42,...,4.16,2.15,31.81,17.37,1,RC Toulon,Union Bordeaux-Bègles,False,False,False
179,2022-2023,2023,26,Section Paloise,Montpellier Hérault Rugby,0.58,0.42,0.55,0.45,0.67,...,4.28,1.42,28.66,8.91,1,Section Paloise,Montpellier Hérault Rugby,False,False,False
180,2022-2023,2023,26,Stade Rochelais,Stade Français Paris,0.55,0.45,0.49,0.51,0.48,...,3.15,1.64,24.75,15.29,1,Stade Rochelais,Stade Français Paris,False,False,False


In [9]:
# Create the dataFrame of the final ranking.
df_classement = pd.DataFrame()
df_classement['Equipe'] = df_predictions['Equipe 1'].drop_duplicates()

# Count the number of times each team appears in the 'Equipe victorieuse' column.
victory_count = df_predictions['Equipe victorieuse'].value_counts()

# Count the number of times the team gets a bonus despite losing.
loser_bonus_count = df_predictions[(df_predictions['Bonus défensif'] == True) | (df_predictions['Bonus offensif perdant'] == True)]['Equipe défaite'].value_counts()

# Count the number of times the team gets a bonus while winning.
winner_bonus_count = df_predictions[(df_predictions['Bonus offensif vainqueur'] == True)]['Equipe victorieuse'].value_counts()

# Add the 3 columns to the DataFrame, filling in with 0 if the team has no bonuses.
df_classement = df_classement.merge(victory_count, how='left', left_on='Equipe', right_index=True)
df_classement = df_classement.merge(winner_bonus_count, how='left', left_on='Equipe', right_index=True).fillna(0)
df_classement = df_classement.merge(loser_bonus_count, how='left', left_on='Equipe', right_index=True).fillna(0)

# Rename the columns for simplicity.
df_classement = df_classement.rename(columns={"Equipe victorieuse_x": "G", "Equipe victorieuse_y": "Bonus gagnant", "Equipe défaite" : "Bonus perdant"})

# Calculate the total number of points. Format.
df_classement["Pts"] = (df_classement["G"]*4 + df_classement["Bonus gagnant"] + df_classement["Bonus perdant"]).astype(int)
df_classement["Bonus"] = (df_classement["Bonus gagnant"] + df_classement["Bonus perdant"]).astype(int)
df_classement["M"] = len(df_classement) * [26]
df_classement = df_classement.drop(columns=['Bonus gagnant', 'Bonus perdant'])
df_classement = df_classement[['Equipe', 'Pts', 'M', 'G', 'Bonus']]

#TOP 6 ranking predicted:
df_classement.nlargest(n=6, columns=['Pts']).reset_index(drop=True)

,Equipe,Pts,M,G,Bonus
0,LOU Rugby,63,26,15,3
1,Stade Français Paris,62,26,14,6
2,Stade Rochelais,61,26,14,5
3,Aviron Bayonnais,58,26,13,6
4,Stade Toulousain,58,26,13,6
5,Racing 92,56,26,13,4


<h3 align="center"> Real 2022-2023's TOP 6 ranking</h3>
<img src ="img/TOP 6 2022-2023.png">

In [10]:
# Global ranking predicted:
df_classement.nlargest(n=14, columns=['Pts']).reset_index(drop=True)

,Equipe,Pts,M,G,Bonus
0,LOU Rugby,63,26,15,3
1,Stade Français Paris,62,26,14,6
2,Stade Rochelais,61,26,14,5
3,Aviron Bayonnais,58,26,13,6
4,Stade Toulousain,58,26,13,6
5,Racing 92,56,26,13,4
6,RC Toulon,56,26,12,8
7,Montpellier Hérault Rugby,56,26,13,4
8,Castres Olympique,55,26,13,3
9,Union Bordeaux-Bègles,53,26,12,5


<h3 align="center"> Real 2022-2023's ranking </h3>
<img src ="img/Classement 2022-2023.png">

<h2 align="center"> Conclusion </h2>
    
With these input data:

    - "Equipe 1",
    - "Equipe 2",
    - "Possession dans son camp Equipe 1",
    - "Possession dans son camp Equipe 2",
    - "Possession dans le camp adverse Equipe 1",
    - "Possession dans le camp adverse Equipe 2",
    - "Possession 22m adverses Equipe 1", 
    - "Possession 22m adverses Equipe 2", 
    - "Mêlées obtenues Equipe 1",
    - "Mêlées obtenues Equipe 2",
    - "Mêlées gagnées Equipe 1",
    - "Mêlées gagnées Equipe 2",
    - "Mêlées perdues Equipe 1",
    - "Mêlées perdues Equipe 2",
    - "Touches obtenues Equipe 1",
    - "Touches obtenues Equipe 2",
    - "Touches gagnées sur son propre lancer Equipe 1",
    - "Touches gagnées sur son propre lancer Equipe 2",
    - "Touches gagnées sur lancer adverse Equipe 1",
    - "Touches gagnées sur lancer adverse Equipe 2",
    - "En-avant commis Equipe 1",
    - "En-avant commis Equipe 2",
    - "Pénalités réussies Equipe 1",
    - "Pénalités réussies Equipe 2",
    - "Pénalités concédées Equipe 1",
    - "Pénalités concédées Equipe 2",
    - "Plaquages réussis Equipe 1",
    - "Plaquages réussis Equipe 2",
    - "Plaquages offensifs réussis Equipe 1",
    - "Plaquages offensifs réussis Equipe 2",
    - "Plaquages manqués Equipe 1",
    - "Plaquages manqués Equipe 2",
    - "Ballons joués au pied Equipe 1",
    - "Ballons joués au pied Equipe 2",
    - "Ballons passés Equipe 1",
    - "Ballons passés Equipe 2"

the models are able to predict the number of tries scored by each team and the winning team fairly well.

However, the models are not accurate at predicting scores. This is where we find the biggest gap between prediction and reality. 
The allocation of bonuses is therefore distorted, as is the final ranking.

Despite everything, the predicted end-of-season rankings for 2022 - 2023 are fairly similar to the actual rankings, especially the TOP 6 (with the exception of UBB).


The models do not take under consideration the players in the team (fitness, injuries, etc.)